## Benchmarking with Argo Worfklows & Vegeta

In this notebook we will dive into how you can run bench marking with batch processing with Argo Workflows, Seldon Core and Vegeta.

Dependencies:

* Seldon core installed as per the docs with Istio as an ingress 
* Argo Workfklows installed in cluster (and argo CLI for commands)


## Setup

### Install Seldon Core
Use the notebook to [set-up Seldon Core with Ambassador or Istio Ingress](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html).

Note: If running with KIND you need to make sure do follow [these steps](https://github.com/argoproj/argo/issues/2376#issuecomment-595593237) as workaround to the `/.../docker.sock` known issue.


### Install Argo Workflows
You can follow the instructions from the official [Argo Workflows Documentation](https://github.com/argoproj/argo#quickstart).

You also need to make sure that argo has permissions to create seldon deployments - for this you can just create a default-admin rolebinding as follows:

In [302]:
!kubectl create namespace argo || echo "namespace already created"
!kubectl apply -n argo -f https://raw.githubusercontent.com/argoproj/argo/v3.1.0/manifests/install.yaml
!kubectl rollout status -n argo deployment/argo-server
!kubectl rollout status -n argo deployment/workflow-controller

Error from server (AlreadyExists): namespaces "argo" already exists
namespace already created
customresourcedefinition.apiextensions.k8s.io/clusterworkflowtemplates.argoproj.io unchanged
customresourcedefinition.apiextensions.k8s.io/cronworkflows.argoproj.io unchanged
customresourcedefinition.apiextensions.k8s.io/workfloweventbindings.argoproj.io unchanged
customresourcedefinition.apiextensions.k8s.io/workflows.argoproj.io unchanged
customresourcedefinition.apiextensions.k8s.io/workflowtemplates.argoproj.io unchanged
serviceaccount/argo unchanged
serviceaccount/argo-server unchanged
role.rbac.authorization.k8s.io/argo-role unchanged
clusterrole.rbac.authorization.k8s.io/argo-aggregate-to-admin unchanged
clusterrole.rbac.authorization.k8s.io/argo-aggregate-to-edit unchanged
clusterrole.rbac.authorization.k8s.io/argo-aggregate-to-view unchanged
clusterrole.rbac.authorization.k8s.io/argo-cluster-role unchanged
clusterrole.rbac.authorization.k8s.io/argo-server-cluster-role unchanged
rolebi

In [5]:
!kubectl create rolebinding default-admin --clusterrole=admin --serviceaccount=default:default

rolebinding.rbac.authorization.k8s.io/default-admin created


### Create Benchmark Argo Workflow

In order to create a benchmark, we created a simple argo workflow template so you can leverage the power of the helm charts.

Before we dive into the contents of the full helm chart, let's first give it a try with some of the settings.

We will run a batch job that will set up a Seldon Deployment with 1 replicas and 4 cpus (with 100 max workers) to send requests.

In [490]:
!helm template seldon-benchmark-workflow helm-charts/seldon-benchmark-workflow/ \
    --set workflow.name=seldon-benchmark-process \
    --set workflow.parallelism=3 \
    --set seldonDeployment.name=sklearn \
    --set seldonDeployment.replicas="1|2" \
    --set seldonDeployment.serverWorkers="2|5|10" \
    --set seldonDeployment.serverThreads=1 \
    --set seldonDeployment.modelUri="gs://seldon-models/sklearn/iris" \
    --set seldonDeployment.server="SKLEARN_SERVER" \
    --set seldonDeployment.apiType="rest|grpc" \
    --set seldonDeployment.requests.cpu="500Mi|1000Mi|2000Mi" \
    --set seldonDeployment.requests.limits="2000Mi" \
    --set seldonDeployment.disableOrchestrator="true|false" \
    --set benchmark.cpu="5" \
    --set benchmark.concurrency="1" \
    --set benchmark.duration="30s" \
    --set benchmark.rate=0 \
    --set benchmark.data='\{"data": {"ndarray": [[0\,1\,2\,3]]\}\}' \
    | argo submit -

Name:                seldon-benchmark-process
Namespace:           default
ServiceAccount:      default
Status:              Pending
Created:             Sat Jun 26 18:39:08 +0100 (now)


In [491]:
!argo list

NAME                       STATUS    AGE   DURATION   PRIORITY
seldon-benchmark-process   Running   2s    2s         0


In [ ]:
!argo logs -f seldon-benchmark-process

seldon-benchmark-process-635956972: [{"name": "sklearn-0", "replicas": "1", "serverWorkers": "2", "serverThreads": "1", "modelUri": "gs://seldon-models/sklearn/iris", "image": "", "server": "SKLEARN_SERVER", "apiType": "rest", "requestsCpu": "500Mi", "requestsMemory": "100Mi", "limitsCpu": "50m", "limitsMemory": "1000Mi", "benchmarkCpu": "5", "concurrency": "1", "duration": "30s", "rate": "0", "disableOrchestrator": "true", "params": "{\"name\": \"sklearn-0\", \"replicas\": \"1\", \"serverWorkers\": \"2\", \"serverThreads\": \"1\", \"modelUri\": \"gs://seldon-models/sklearn/iris\", \"image\": \"\", \"server\": \"SKLEARN_SERVER\", \"apiType\": \"rest\", \"requestsCpu\": \"500Mi\", \"requestsMemory\": \"100Mi\", \"limitsCpu\": \"50m\", \"limitsMemory\": \"1000Mi\", \"benchmarkCpu\": \"5\", \"concurrency\": \"1\", \"duration\": \"30s\", \"rate\": \"0\", \"disableOrchestrator\": \"true\"}"}, {"name": "sklearn-1", "replicas": "1", "serverWorkers": "2", "serverThreads": "1", "modelUri": "gs:

seldon-benchmark-process-3399506974: time="2021-06-26T17:39:23.062Z" level=info msg="Starting Workflow Executor" executorType= version=untagged
seldon-benchmark-process-3399506974: time="2021-06-26T17:39:23.066Z" level=info msg="Creating a docker executor"
seldon-benchmark-process-3399506974: time="2021-06-26T17:39:23.066Z" level=info msg="Executor initialized" deadline="0001-01-01 00:00:00 +0000 UTC" includeScriptOutput=false namespace=default podName=seldon-benchmark-process-3399506974 template="{\"name\":\"create-seldon-resource-template\",\"inputs\":{\"parameters\":[{\"name\":\"inparam\",\"value\":\"sklearn-0\"},{\"name\":\"replicas\",\"value\":\"1\"},{\"name\":\"serverWorkers\",\"value\":\"2\"},{\"name\":\"serverThreads\",\"value\":\"1\"},{\"name\":\"modelUri\",\"value\":\"gs://seldon-models/sklearn/iris\"},{\"name\":\"image\",\"value\":\"\"},{\"name\":\"server\",\"value\":\"SKLEARN_SERVER\"},{\"name\":\"apiType\",\"value\":\"rest\"},{\"name\":\"requestsCpu\",\"value\":\"500Mi\"},

seldon-benchmark-process-1959190040: Waiting for deployment "sklearn-0-default-0-classifier" rollout to finish: 0 of 1 updated replicas are available...
seldon-benchmark-process-2980713737: Waiting for deployment "sklearn-1-default-0-classifier" rollout to finish: 0 of 1 updated replicas are available...
seldon-benchmark-process-1959190040: deployment "sklearn-0-default-0-classifier" successfully rolled out
seldon-benchmark-process-2980713737: deployment "sklearn-1-default-0-classifier" successfully rolled out
seldon-benchmark-process-674182850: Waiting for deployment "sklearn-2-default-0-classifier" rollout to finish: 0 of 1 updated replicas are available...
seldon-benchmark-process-674182850: deployment "sklearn-2-default-0-classifier" successfully rolled out
seldon-benchmark-process-4293706209: {"latencies":{"total":29971292400,"mean":6568330,"50th":6142206,"90th":8891155,"95th":10257702,"99th":13721939,"max":47531100,"min":2869100},"bytes_in":{"total":835029,"mean":183},"bytes_out"

seldon-benchmark-process-1457865277: Waiting for deployment "sklearn-3-default-0-classifier" rollout to finish: 0 of 1 updated replicas are available...
seldon-benchmark-process-3754965750: Waiting for deployment "sklearn-4-default-0-classifier" rollout to finish: 0 of 1 updated replicas are available...
seldon-benchmark-process-1457865277: deployment "sklearn-3-default-0-classifier" successfully rolled out
seldon-benchmark-process-3754965750: deployment "sklearn-4-default-0-classifier" successfully rolled out
seldon-benchmark-process-2417483003: Waiting for deployment "sklearn-5-default-0-classifier" rollout to finish: 0 of 1 updated replicas are available...
seldon-benchmark-process-2417483003: deployment "sklearn-5-default-0-classifier" successfully rolled out
seldon-benchmark-process-2335221500: {"latencies":{"total":29976500600,"mean":5253505,"50th":5152429,"90th":6549656,"95th":7157637,"99th":9006746,"max":36220900,"min":2714000},"bytes_in":{"total":1044198,"mean":183},"bytes_out

seldon-benchmark-process-4178145511: deployment "sklearn-7-default-0-classifier" successfully rolled out
seldon-benchmark-process-1920257888: deployment "sklearn-8-default-0-classifier" successfully rolled out
seldon-benchmark-process-3599782625: {"date":"2021-06-26T17:44:05Z","endReason":"timeout","options":{"host":"istio-ingressgateway.istio-system.svc.cluster.local:80","proto":"/proto/prediction.proto","import-paths":["/proto","."],"call":"seldon.protos.Seldon/Predict","insecure":true,"total":2147483647,"concurrency":1,"connections":1,"duration":30000000000,"timeout":20000000000,"dial-timeout":10000000000,"data":{"data":{"ndarray":[[0,1,2,3]]}},"binary":false,"metadata":{"namespace":"default","seldon":"sklearn-6"},"CPUs":5},"count":7214,"total":30001136300,"average":4032255,"fastest":2482200,"slowest":47126600,"rps":240.4575589358594,"errorDistribution":{"rpc error: code = Unavailable desc = transport is closing":1},"statusCodeDistribution":{"OK":7213,"Unavailable":1},"latencyDistri

...

seldon-benchmark-process-3169575794: time="2021-06-26T17:56:22.718Z" level=info msg="Starting Workflow Executor" executorType= version=untagged
seldon-benchmark-process-3169575794: time="2021-06-26T17:56:22.721Z" level=info msg="Creating a docker executor"
seldon-benchmark-process-3169575794: time="2021-06-26T17:56:22.722Z" level=info msg="Executor initialized" deadline="0001-01-01 00:00:00 +0000 UTC" includeScriptOutput=false namespace=default podName=seldon-benchmark-process-3169575794 template="{\"name\":\"create-seldon-resource-template\",\"inputs\":{\"parameters\":[{\"name\":\"inparam\",\"value\":\"sklearn-31\"},{\"name\":\"replicas\",\"value\":\"1\"},{\"name\":\"serverWorkers\",\"value\":\"10\"},{\"name\":\"serverThreads\",\"value\":\"1\"},{\"name\":\"modelUri\",\"value\":\"gs://seldon-models/sklearn/iris\"},{\"name\":\"image\",\"value\":\"\"},{\"name\":\"server\",\"value\":\"SKLEARN_SERVER\"},{\"name\":\"apiType\",\"value\":\"grpc\"},{\"name\":\"requestsCpu\",\"value\":\"500Mi\"

seldon-benchmark-process-1796419992: {"date":"2021-06-26T17:57:29Z","endReason":"timeout","options":{"host":"istio-ingressgateway.istio-system.svc.cluster.local:80","proto":"/proto/prediction.proto","import-paths":["/proto","."],"call":"seldon.protos.Seldon/Predict","insecure":true,"total":2147483647,"concurrency":1,"connections":1,"duration":30000000000,"timeout":20000000000,"dial-timeout":10000000000,"data":{"data":{"ndarray":[[0,1,2,3]]}},"binary":false,"metadata":{"namespace":"default","seldon":"sklearn-30"},"CPUs":5},"count":5531,"total":30000575800,"average":5255882,"fastest":2767800,"slowest":75637700,"rps":184.36312812369422,"errorDistribution":{"rpc error: code = Unavailable desc = transport is closing":1},"statusCodeDistribution":{"OK":5530,"Unavailable":1},"latencyDistribution":[{"percentage":10,"latency":3808900},{"percentage":25,"latency":4293900},{"percentage":50,"latency":4942000},{"percentage":75,"latency":5698700},{"percentage":90,"latency":6705100},{"percentage":95,"l

seldon-benchmark-process-1029786471: time="2021-06-26T17:58:10.621Z" level=info msg="Resource: default/seldondeployment.machinelearning.seldon.io/sklearn-34. SelfLink: apis/machinelearning.seldon.io/v1/namespaces/default/seldondeployments/sklearn-34"
seldon-benchmark-process-1029786471: time="2021-06-26T17:58:10.622Z" level=info msg="No output parameters"
seldon-benchmark-process-1843931402: {"date":"2021-06-26T17:58:24Z","endReason":"timeout","options":{"host":"istio-ingressgateway.istio-system.svc.cluster.local:80","proto":"/proto/prediction.proto","import-paths":["/proto","."],"call":"seldon.protos.Seldon/Predict","insecure":true,"total":2147483647,"concurrency":1,"connections":1,"duration":30000000000,"timeout":20000000000,"dial-timeout":10000000000,"data":{"data":{"ndarray":[[0,1,2,3]]}},"binary":false,"metadata":{"namespace":"default","seldon":"sklearn-32"},"CPUs":5},"count":5953,"total":30000375700,"average":4875132,"fastest":2568700,"slowest":54382000,"rps":198.43084831767624,"

seldon-benchmark-process-3451257566: deployment "sklearn-35-default-0-classifier" successfully rolled out
seldon-benchmark-process-2777615208: seldondeployment.machinelearning.seldon.io "sklearn-33" deleted
seldon-benchmark-process-757849581: seldondeployment.machinelearning.seldon.io "sklearn-34" deleted
seldon-benchmark-process-4093479683: time="2021-06-26T17:59:56.390Z" level=info msg="Starting Workflow Executor" executorType= version=untagged
seldon-benchmark-process-4093479683: time="2021-06-26T17:59:56.395Z" level=info msg="Creating a docker executor"
seldon-benchmark-process-4093479683: time="2021-06-26T17:59:56.395Z" level=info msg="Executor initialized" deadline="0001-01-01 00:00:00 +0000 UTC" includeScriptOutput=false namespace=default podName=seldon-benchmark-process-4093479683 template="{\"name\":\"create-seldon-resource-template\",\"inputs\":{\"parameters\":[{\"name\":\"inparam\",\"value\":\"sklearn-36\"},{\"name\":\"replicas\",\"value\":\"2\"},{\"name\":\"serverWorkers\",

seldon-benchmark-process-1696573853: Waiting for deployment "sklearn-38-default-0-classifier" rollout to finish: 0 of 2 updated replicas are available...
seldon-benchmark-process-1696573853: Waiting for deployment "sklearn-38-default-0-classifier" rollout to finish: 1 of 2 updated replicas are available...
seldon-benchmark-process-1696573853: deployment "sklearn-38-default-0-classifier" successfully rolled out
seldon-benchmark-process-2282414864: {"latencies":{"total":29973960000,"mean":6531697,"50th":5920169,"90th":8812066,"95th":10202509,"99th":13490529,"max":139571700,"min":3613900},"bytes_in":{"total":839787,"mean":183},"bytes_out":{"total":156026,"mean":34},"earliest":"2021-06-26T18:00:45.9028724Z","latest":"2021-06-26T18:01:15.9060677Z","end":"2021-06-26T18:01:15.9109733Z","duration":30003195300,"wait":4905600,"requests":4589,"rate":152.9503759221272,"throughput":152.92537222840383,"success":1,"status_codes":{"200":4589},"errors":[],"params":{"name":"sklearn-36","replicas":"2","s

In [ ]:
!argo get seldon-benchmark-process

## Process the results

We can now print the results in a consumable format.

In [484]:
def get_results(results, print_results=True):
    final = {}
    # For GHZ / grpc
    if "average" in results:
        final["mean"] = results["average"] / 1e6
        if results.get("latencyDistribution", False):
            final["50th"] = results["latencyDistribution"][-5]["latency"] / 1e6
            final["90th"] = results["latencyDistribution"][-3]["latency"] / 1e6
            final["95th"] = results["latencyDistribution"][-2]["latency"] / 1e6
            final["99th"] = results["latencyDistribution"][-1]["latency"] / 1e6
        final["throughputAchieved"] = results["rps"]
        final["success"] = results["statusCodeDistribution"].get("OK", 0)
        final["errors"] = sum(results["statusCodeDistribution"].values()) - final["success"]
    # For vegeta / rest
    else:
        final["mean"] = results["latencies"]["mean"] / 1e6
        final["50th"] = results["latencies"]["50th"] / 1e6
        final["90th"] = results["latencies"]["90th"] / 1e6
        final["95th"] = results["latencies"]["95th"] / 1e6
        final["99th"] = results["latencies"]["99th"] / 1e6
        final["throughputAchieved"] = results["throughput"]
        final["success"] = results["status_codes"].get("200", 0)
        final["errors"] = sum(results["status_codes"].values()) - final["success"]
    for k in results["params"].keys():
        final[k] = results["params"][k]
    if print_results:
        print("-----")
        print("ParamNames:", results["params"].keys())
        print("ParamNames:", results["params"].values())
        print("\tLatencies:")
        print("\t\tmean:", final["mean"], "ms")
        print("\t\t50th:", final["50th"], "ms")
        print("\t\t90th:", final["90th"], "ms")
        print("\t\t95th:", final["95th"], "ms")
        print("\t\t99th:", final["99th"], "ms")
        print("")
        print("\tRate:", str(final["throughputAchieved"]) + "/s")
        print("\tSuccess:", final["success"])
        print("\tErrors:", final["errors"])
    return final

In [485]:
wf_logs = !argo logs --no-color seldon-benchmark-process 
wf_bench = [json.loads(":".join(w.split(":")[1:])) for w in wf_logs if "latenc" in w]
print(wf_bench)

[{'latencies': {'total': 29982933100, 'mean': 5815153, '50th': 5555652, '90th': 7956292, '95th': 8902790, '99th': 11839561, 'max': 24303300, 'min': 2942900}, 'bytes_in': {'total': 943548, 'mean': 183}, 'bytes_out': {'total': 175304, 'mean': 34}, 'earliest': '2021-06-26T17:26:09.1228728Z', 'latest': '2021-06-26T17:26:39.1286544Z', 'end': '2021-06-26T17:26:39.1350149Z', 'duration': 30005781600, 'wait': 6360500, 'requests': 5156, 'rate': 171.8335509047363, 'throughput': 171.7971340672814, 'success': 1, 'status_codes': {'200': 5156}, 'errors': [], 'params': {'name': 'sklearn-0', 'replicas': '1', 'serverWorkers': '2', 'serverThreads': '1', 'modelUri': 'gs://seldon-models/sklearn/iris', 'image': '', 'server': 'SKLEARN_SERVER', 'apiType': 'rest', 'requestsCpu': '500Mi', 'requestsMemory': '100Mi', 'limitsCpu': '50m', 'limitsMemory': '1000Mi', 'benchmarkCpu': '5', 'concurrency': '1', 'duration': '30s', 'rate': '0', 'disableOrchestrator': 'false'}}, {'date': '2021-06-26T17:26:52Z', 'endReason': 

In [486]:
results = []
for w in wf_bench:
    # Prints the results in a consumable format
    results.append(get_results(w))

-----
ParamNames: dict_keys(['name', 'replicas', 'serverWorkers', 'serverThreads', 'modelUri', 'image', 'server', 'apiType', 'requestsCpu', 'requestsMemory', 'limitsCpu', 'limitsMemory', 'benchmarkCpu', 'concurrency', 'duration', 'rate', 'disableOrchestrator'])
ParamNames: dict_values(['sklearn-0', '1', '2', '1', 'gs://seldon-models/sklearn/iris', '', 'SKLEARN_SERVER', 'rest', '500Mi', '100Mi', '50m', '1000Mi', '5', '1', '30s', '0', 'false'])
	Latencies:
		mean: 5.815153 ms
		50th: 5.555652 ms
		90th: 7.956292 ms
		95th: 8.90279 ms
		99th: 11.839561 ms

	Rate: 171.7971340672814/s
	Success: 5156
	Errors: 0
-----
ParamNames: dict_keys(['name', 'replicas', 'serverWorkers', 'serverThreads', 'modelUri', 'image', 'server', 'apiType', 'requestsCpu', 'requestsMemory', 'limitsCpu', 'limitsMemory', 'benchmarkCpu', 'concurrency', 'duration', 'rate', 'disableOrchestrator'])
ParamNames: dict_values(['sklearn-1', '1', '2', '1', 'gs://seldon-models/sklearn/iris', '', 'SKLEARN_SERVER', 'grpc', '500Mi'

In [489]:
!argo delete seldon-benchmark-process || echo "Argo workflow already deleted or not exists"

Workflow 'seldon-benchmark-process' not found


In [ ]:
!kubectl delete -n argo -f https://raw.githubusercontent.com/argoproj/argo/v3.1.0/manifests/install.yaml

## Deeper Analysis
Now that we have all the parameters, we can do a deeper analysis

In [488]:
import pandas as pd

df = pd.DataFrame.from_dict(results)
df.head()

,mean,50th,90th,95th,99th,throughputAchieved,success,errors,name,replicas,...,apiType,requestsCpu,requestsMemory,limitsCpu,limitsMemory,benchmarkCpu,concurrency,duration,rate,disableOrchestrator
0,5.815153,5.555652,7.956292,8.902790,11.839561,171.797134,5156,0,sklearn-0,1,...,rest,500Mi,100Mi,50m,1000Mi,5,1,30s,0,false
1,6.085275,5.836000,7.814800,8.694900,11.005800,158.389138,4751,1,sklearn-1,1,...,grpc,500Mi,100Mi,50m,1000Mi,5,1,30s,0,false
2,6.528934,6.240889,8.269833,9.151776,11.575048,152.982098,4591,0,sklearn-2,2,...,rest,500Mi,100Mi,50m,1000Mi,5,1,30s,0,false
3,5.004515,4.676200,6.748600,7.604800,9.469800,193.429957,5802,1,sklearn-3,2,...,grpc,500Mi,100Mi,50m,1000Mi,5,1,30s,0,false
